In [ ]:
#import all the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)



tf.keras.backend.set_floatx('float64')


tf.autograph.set_verbosity(0)


## **Customer Churn Prediction Model**
Using the factors correlated with customer status above, a prediction model ise build, aiming to predict the potential of customers who are about to churn away of company and have the appropriate respond before their churn decision. 

Three factors used are: Number of Referrals, Tenure in Months, Contract, Online Security, PRemium Tech Support. Among these factors, Online Security and Premium Tech Support are two services that have the high impact with customer's churn decision. These factors are binary variable (Yes as 1 and No as 0) while Contract is also classfified variable (Month to Month as 1, 1 Year as 1 and 2 Yeats as 2) Other factors are numerous variables. Target variable is Customer Status. To make it easier when building a model, we eliminate customers whose customer status is Joined since there are other factors outside the collected dataset affecting the decision of joining of these customers. 




In [ ]:
tele = pd.read_csv('telecom_customer_churn.csv')
tele_copy = tele.copy()
#Encoding categorical answers
service_anw = {'Yes' : 1, 'No':0,'Cable':0, 'Fiber Optic' :1, 'DSL': 2}
contract = {'Month-to-Month' : 0, 'One Year': 1, 'Two Year':2}
customer_status = {'Stayed' : 0, 'Churned':1, 'Joined': 2}
payment_method = {'Credit Card': 0, 'Bank Withdrawal':1, 'Mailed Check':2}
service = ['Phone Service', 'Multiple Lines', 'Internet Service', 
           'Internet Type', 'Online Security', 'Online Backup', 'Device Protection Plan',
           'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Streaming Music',
           'Unlimited Data']
for i in range(len(service)):
  tele_copy[service[i]].replace(service_anw, inplace=True)
tele_copy['Customer Status'].replace(customer_status, inplace=True)
tele_copy['Contract'].replace(contract, inplace=True)
tele_copy['Payment Method'].replace(payment_method, inplace = True)

In [ ]:
#We assume all of the null data from services are due to customers not subcribing company's service
#then we filla all the null data with 0 refering to no subscription to service
tele_copy = tele_copy.fillna(0)
tele_copy = tele_copy[(tele_copy['Customer Status']==0) |
                      (tele_copy['Customer Status']==1)]

### **1. Neural Network Model**
Using Tensorflow and Keras to build neural network model. 



In [ ]:
cols = ['Number of Referrals', 'Tenure in Months', 'Online Security', 'Premium Tech Support','Contract']
x_tele = tele_copy[cols]
y_tele = tele_copy['Customer Status']
print('X.shape', x_tele.shape, 'y.shape', y_tele.shape)

#Split the data using sklearn routine
X_train, X_, y_train, y_ = train_test_split(x_tele, y_tele, test_size = 0.40, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size = 0.50, random_state=1)
print('X_train.shape', X_train.shape, 'y_train', y_train.shape)
print('X_cv', X_cv.shape, 'y_cv', y_cv.shape)
print('X_test', X_test.shape, 'y_test', y_test.shape )

X.shape (6589, 5) y.shape (6589,)
X_train.shape (3953, 5) y_train (3953,)
X_cv (1318, 5) y_cv (1318,)
X_test (1318, 5) y_test (1318,)


In [ ]:
X_train.head(5)

,Number of Referrals,Tenure in Months,Online Security,Premium Tech Support,Contract
3203,0,1,0.0,0.0,0
856,2,61,0.0,0.0,2
3590,1,58,1.0,0.0,2
5678,0,1,0.0,0.0,0
3469,9,70,1.0,0.0,2


In [ ]:
#Create neural network model
model = Sequential(
    [Dense(units=120, activation = 'relu'),
     Dense(units=40, activation = 'relu'),
     Dense(units=8, activation = 'relu'),
     Dense(units=1, activation = 'sigmoid')], name = 'Model_1'
)
model.compile(loss = 'binary_crossentropy',
               optimizer= Adam(learning_rate=0.01))
model.fit(X_train, y_train, epochs = 1000)

Epoch 1/1000
124/124 [==============================] - 1s 2ms/step - loss: 0.4267
Epoch 2/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3602
Epoch 3/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3515
Epoch 4/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3555
Epoch 5/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3493
Epoch 6/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3414
Epoch 7/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3447
Epoch 8/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3405
Epoch 9/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3393
Epoch 10/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3363
Epoch 11/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3334
Epoch 12/1000
124/124 [==============================] - 0s 2ms/step - loss: 0.3400
E

In [ ]:
model.summary()

Model: "Model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               720       
                                                                 
 dense_1 (Dense)             (None, 40)                4840      
                                                                 
 dense_2 (Dense)             (None, 8)                 328       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 5,897
Trainable params: 5,897
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def eval_cat_err(y, yhat):
    """ 
    Calculate the categorization error
    Args:
      y    : (ndarray  Shape (m,) or (m,1))  target value of each example
      yhat : (ndarray  Shape (m,) or (m,1))  predicted value of each example
    Returns:|
      cerr: (scalar)             
    """
    m = len(y)
    incorrect = 0
    for i in range(m):
        if yhat[i] != y[i]:
            incorrect = incorrect +1
    cerr = incorrect/m
    return(cerr)

In [ ]:
y_train_hat = (model.predict(X_train)>0.5).astype(int)
y_train_hat.reshape((1,3953))
y_train = np.array(y_train)


124/124 [==============================] - 0s 2ms/step


In [ ]:
train_cerr_complex = eval_cat_err(y_train, y_train_hat)
print(f"categorization error, training: {train_cerr_complex:0.5f}")

categorization error, training: 0.16140


In [ ]:
y_cv_hat = (model.predict(X_cv)>0.5).astype(int)
y_cv_hat.reshape((1, 1318))
y_cv = np.array(y_cv)

42/42 [==============================] - 0s 2ms/step


In [ ]:
cv_cerr_complex = eval_cat_err(y_cv, y_cv_hat)
print(f"categorization error, cv: {train_cerr_complex:0.5f}")

categorization error, cv: 0.16140


### **2. Logistics Regression**


In [ ]:
#Logistics Regression using Scikit-Learn
from sklearn.linear_model import LogisticRegression

lr_model= LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
acc_train = lr_model.score(X_train, y_train)
acc_train

0.8039463698456868

In [ ]:
acc_cv = lr_model.score(X_cv, y_cv)
print("acc_cv:", acc_cv)

acc_cv: 0.7860394537177542


### **3. Random Forest**

In [ ]:
#random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor()

In [ ]:
np.mean(cross_val_score(rf, X_train, y_train, scoring = "neg_mean_absolute_error", cv=3))

-0.2163681786305707

In [ ]:
np.mean(cross_val_score(rf, X_cv, y_cv, scoring = "neg_mean_absolute_error", cv=3))

-0.23449536235221213

In [ ]:
# tune models Gridsearch CV: find the decision tree model with best result (least error)
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators': range(10,300,10), 'criterion': ('mse', 'mae'), 'max_features': ('auto', 'sqrt', 'log2')}
gs = GridSearchCV(rf, parameters, scoring = 'neg_mean_absolute_error', cv=3)
gs.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was d

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'criterion': ('mse', 'mae'),
                         'max_features': ('auto', 'sqrt', 'log2'),
                         'n_estimators': range(10, 300, 10)},
             scoring='neg_mean_absolute_error')

In [ ]:
gs.best_score_

-0.2117270833645388

In [ ]:
gs.best_estimator_

RandomForestRegressor(criterion='mae', max_features='log2', n_estimators=20)

In [ ]:
#test ensembles
tpred_rf = (gs.best_estimator_.predict(X_test)>0.5).astype(int) #random forest
tpred_nn = (model.predict(X_test)>0.5).astype(int) #neural network
tpred_lr = lr_model.predict(X_test) #logistic regression 


42/42 [==============================] - 0s 1ms/step


In [35]:
from sklearn.metrics import mean_absolute_error
print('Mean Absolute Error of Logistic Regression Model:',mean_absolute_error(y_test, tpred_lr))
print('Mean Absolute Error of Random Forest Model:', mean_absolute_error(y_test, tpred_rf))
print('Mean Absolute Error of Neural Network Model:',mean_absolute_error(y_test, tpred_nn))


Mean Absolute Error of Logistic Regression Model: 0.20409711684370258
Mean Absolute Error of Random Forest Model: 0.19802731411229135
Mean Absolute Error of Neural Network Model: 0.16919575113808802


### **4. Conclusion**
We built three models to predict customer's decision: whether or not the churn decision is made. Those three models are Logistic Regression, Random Forest and Neural Network. They have mean absolute error as 0.204, 0.198, 0.169 respectively. Based on the error of each model, to make the most accurate prediction of customers' churn decision, we use neural network model for further deployment because it generates the lowest error. 

We are using five features that are highly correlated with customer's churn decision from data analysis part. To make more accurate prediction, we can collect more data from customers as well as find more related features that can have effect of customer's decision. 